In [2]:
import pandas as pd
from datetime import datetime
from gurobipy import *
import numpy as np
import time

In [3]:
df = pd.read_excel('./yellow_tripdata_130101.xlsx')

In [4]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID
0,2013-01-01 00:00:00,2013-01-01 00:04:00,1.71,236,162
1,2013-01-01 00:00:00,2013-01-01 00:19:00,8.48,162,244
2,2013-01-01 00:00:00,2013-01-01 00:11:00,9.35,138,216
3,2013-01-01 00:00:00,2013-01-01 00:15:00,2.56,144,170
4,2013-01-01 00:00:00,2013-01-01 00:16:00,4.05,249,239


In [5]:
df['trip_distance'] = pd.to_numeric(df['trip_distance'], errors = 'coerce')
avg_distance_df = df.groupby(['PULocationID', 'DOLocationID'])['trip_distance'].mean().reset_index()
avg_distance_dict = {(row['PULocationID'], row['DOLocationID']): row['trip_distance'] for _, row in avg_distance_df.iterrows()}

In [6]:
## num: total number of demand
## c: objective function cost
## x: decision variable
def sol_pro(c, F, pick_up_time, deliver_time, travel_dist, relocation_time, buffer_time=99999, print_info=False): #, actual_dist, time_buffer, dist_buffer):
    # construct matrix m (reachable demand)
    num=len(pick_up_time)
    m_mat=np.zeros((2*num+2, 2*num+2),dtype=int)
    m_mat[0:num,num:2*num]=np.eye(num, dtype=int)
    m_mat[num:2*num,0:num]=1-np.identity(num)
    m_mat[num:2*num,2*num+1]=1
    m_mat[2*num,0:num]=1
    m_mat[2*num,2*num+1]=F
    ## relocation time 
    #relocation_time=np.zeros((num,num))
    #delta=np.zeros((num,num))
    #eta=np.zeros((num,num))
    #relocation_dist=np.zeros((num,num))
    ## relocation time check
    for i in range(num):
        for j in range(num):
            if i==j:
                continue;
    #        eta[i,j]=0.5*(travel_dist[i]/actual_dist[i,num+i]+travel_dist[j]/actual_dist[j,num+j])
    #        relocation_dist[i,j]=actual_dist[num+i,j]*eta[i,j]
    #        delta[i,j]=0.5*((deliver_time[i]-pick_up_time[i])/travel_dist[i]+(deliver_time[j]-pick_up_time[j])/travel_dist[j])
    #        relocation_time[i,j]=relocation_dist[i,j]*delta[i,j]
            if((pick_up_time[j] - deliver_time[i]).total_seconds()<relocation_time[i,j]):
                m_mat[num+i,j]=0
    #       if(relocation_time>buffer_time):
    #           m_mat[num+i,num+j]=0
    #        if(relocation_dist>dist_buffer):
    #            m_mat[num+i,num+j]=0
    
    ##model 1
    model_1 = Model("lp_model")
    model_1.Params.LogToConsole = 0
    x = model_1.addMVar((2*num+2,2*num+2), lb=0, vtype = GRB.CONTINUOUS, name = "x")
    model_1.update()
    
    ## set objective functions
    model_1.setObjective(quicksum(c[i, j]*x[i][j] for i in range(2*num+2) for j in range(2*num+2))
                         , GRB.MINIMIZE)
    
    ## constraint
    model_1.addConstrs((x[i][j]<=m_mat[i,j] for i in range(2*num+2) for j in range(2*num+2)), "Link capacity constraint")
    model_1.addConstrs((quicksum(x[j][i] for j in range(2*num+2)) - quicksum(x[i][j] for j in range(2*num+2)) == 0 for i in range(2*num)), "balance_1")
    model_1.addConstr((quicksum(x[2*num][j] for j in [i for i in range(2*num+2) if i != 2*num]) - F == 0), "balance_2")
    model_1.addConstr((quicksum(x[j][2*num+1] for j in range(2*num+1)) - F == 0), "balance_3")
#     model_1.addConstr(x[22, 0] == 1, "test")

    ###
    ## solve and result
    model_1.optimize()
    ## solution for decision variables
    # print(x)
    # print(x[0:num, num:2*num])
    ## solution for objective values
    # print('Obj: %g' % model_1.objVal)
    # print('objective value =', model_1.objVal)
    if print_info:
        for var in model_1.getVars():
            if var.x >= 1:
                print(var.varName, '=', var.x)

    ## get every car chain
    t_value=[]
    for var in model_1.getVars():
        t_value.append(var.x)
    chain=[]
    ## collect how many car need to use and the first state for each car 
    for i in range((2*num+2)*2*num, (2*num+2)*2*num+num):
        if t_value[i]==1:
            temp=[]
            temp.append(i-(2*num+2)*2*num+1)
            chain.append(temp)
    ## use for finish all chain
    for i in range(len(chain)):
        temp=chain[i][0]
        next_place=-1
        while next_place!='d':
            for j in range((num-1+temp)*(2*num+2), (num-1+temp)*(2*num+2)+num):
                if t_value[j]==1:
                    next_place=j-(num-1+temp)*(2*num+2)+1
            if t_value[(num+temp)*(2*num+2)-1]==1:
                next_place='d'
            chain[i].append(next_place)
            temp=next_place
    return chain, model_1.objVal

In [38]:
def add_ord(current_chain, new_pick_up_time, new_deliver_time, pick_up_time, deliver_time, s_loc, e_loc, new_s_loc, new_e_loc, dist_dict, relocataion_time ,relocation_cost, fc, dc, current_obj, rev_loss):
    min_cost=fc+dc+dc
    pick_chain="new_chain"
    loc="new"
    for i in range(len(current_chain)):
        for j in range(len(current_chain[i])):
            ## new deliver is earlier than first pick_up ## should consider relocation time
            if j==0:
                if (new_e_loc, s_loc[current_chain[i][j]-1]) in dist_dict:
                    temp_dist=dist_dict[new_e_loc, s_loc[current_chain[i][j]-1]]
                    temp_time=temp_dist*relocation_cost
                    if (pick_up_time[current_chain[i][j]-1] - new_deliver_time).total_seconds() >= temp_time:
                        r_cost = temp_dist * relocation_cost
                    else:
                        r_cost = 99999 * relocation_cost
                else:
                    r_cost = 99999 * relocation_cost
                if r_cost<=min_cost:
                    min_cost=r_cost
                    pick_chain=i
                    loc=j
            ## new pick up is later than last delivery time ## should consider relocation time
            elif j==len(current_chain[i])-1:
                if (e_loc[current_chain[i][j-1]-1], new_s_loc) in dist_dict:
                    temp_dist=dist_dict[e_loc[current_chain[i][j-1]-1], new_s_loc]
                    temp_time=temp_dist*relocation_cost
                    if (new_pick_up_time - deliver_time[current_chain[i][j-1]-1]).total_seconds() >= temp_time:
                        r_cost = temp_dist * relocation_cost
                    else:
                        r_cost = 99999 * relocation_cost
                else:
                    r_cost = 99999 * relocation_cost
                if r_cost<=min_cost:
                    min_cost=r_cost
                    pick_chain=i
                    loc=j
            ## new delivery is earlier than next pickup and new pick up is later than previous 
            elif (j!=0) & (j<len(current_chain[i])-1):
                if ((new_e_loc, s_loc[current_chain[i][j]-1]) in dist_dict) & ((e_loc[current_chain[i][j-1]-1], new_s_loc) in dist_dict):
                    s_temp_dist = dist_dict[e_loc[current_chain[i][j-1]-1], new_s_loc]
                    e_temp_dist = dist_dict[new_e_loc, s_loc[current_chain[i][j]-1]]
                    s_temp_time=s_temp_dist*relocation_cost
                    e_temp_time=e_temp_dist*relocation_cost
                    if ((pick_up_time[current_chain[i][j]-1]- new_deliver_time).total_seconds() >= e_temp_time) & ((new_pick_up_time - deliver_time[current_chain[i][j-1]-1]).total_seconds() >= s_temp_time):
                        r_cost = (e_temp_dist+s_temp_dist) * relocation_cost
                    else: 
                        r_cost = 99999 * relocation_cost *2
                else:
                    r_cost = 99999 * relocation_cost *2
                if r_cost<=min_cost:
                    min_cost=r_cost
                    pick_chain=i
                    loc=j
    if pick_chain=="new_chain" or loc==0 or loc==(len(current_chain[pick_chain])-1):
        current_obj = current_obj + min_cost - rev_loss
    else:
        current_obj = current_obj + min_cost - dist_dict[e_loc[current_chain[pick_chain][loc-1]-1], s_loc[current_chain[pick_chain][loc]-1]] * relocation_cost - rev_loss
    return pick_chain, loc, min_cost, current_obj

In [8]:
def get_all_info(n, fleet_cost, dispatch_cost, s_loc, e_loc, rev_loss, relocation_cost):
    # =====
    MY_LARGE_NUM = 99999
    
    O_ID = 2 * n
    D_ID = 2 * n + 1
    node_set = range(2 * n + 2)
    
    d = {i: dispatch_cost for i in range(n)}
    c = {(i, j): MY_LARGE_NUM for i in node_set for j in node_set}
    relocation_time = {(i, j): MY_LARGE_NUM for i in range(n) for j in range(n)}
    
    for i in node_set:
        for j in node_set:
            if i == O_ID and j == D_ID:
                c[i, j] = 0

            elif i == O_ID:
                if j < n:
                    c[i, j] = fleet_cost + d[j]

            elif j == D_ID:
                if n <= i < 2 * n:
                    c[i, j] = d[i % n]

            elif i < n:
                c[i, n + i] = -rev_loss[i]

            elif j < n:  # now i >= n, indicating i^+
                e_ = e_loc[i % n]  # ending location of order i
                s_ = s_loc[j]      # starting location of order j
                if (e_, s_) in avg_distance_dict:
                    c[i, j] = avg_distance_dict[e_, s_] * relocation_cost
                    relocation_time[i % n, j] = avg_distance_dict[e_, s_] * 30
    
    return c, relocation_time

In [9]:
def get_basic_info(sampled_df):
    n = len(sampled_df)
    s_time = dict(zip(range(n), sampled_df['tpep_pickup_datetime']))
    e_time = dict(zip(range(n), sampled_df['tpep_dropoff_datetime']))
    s_loc = dict(zip(range(n), sampled_df['PULocationID']))
    e_loc = dict(zip(range(n), sampled_df['DOLocationID']))
    rev_loss = dict(zip(range(n), sampled_df['rev_loss']))
    
    return s_time, e_time, s_loc, e_loc, rev_loss

In [10]:
def generate_an_instance(n, unit_rev_loss, fleet_cost, dispatch_cost, relocation_cost):
    # n: number of orders to sample
    # seed = 1001
    # sampled_df = df.sample(n = n, random_state = seed)
    sampled_df = df.sample(n = n)

    sampled_df['tpep_pickup_datetime'] = pd.to_datetime(sampled_df['tpep_pickup_datetime'], format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    sampled_df['tpep_dropoff_datetime'] = pd.to_datetime(sampled_df['tpep_dropoff_datetime'], format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    sampled_df['rev_loss'] = sampled_df['trip_distance'] * unit_rev_loss
    sampled_df = sampled_df.reset_index(drop = True)

    s_time, e_time, s_loc, e_loc, rev_loss = get_basic_info(sampled_df)
    c, relocation_time = get_all_info(n, fleet_cost, dispatch_cost, s_loc, e_loc, rev_loss, relocation_cost)
    
    return c, s_time, e_time, relocation_time, sampled_df

In [11]:
def generate_a_random_order(unit_rev_loss):
    sampled_df = df.sample(n = 1)

    sampled_df['tpep_pickup_datetime'] = pd.to_datetime(sampled_df['tpep_pickup_datetime'], format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    sampled_df['tpep_dropoff_datetime'] = pd.to_datetime(sampled_df['tpep_dropoff_datetime'], format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    sampled_df['rev_loss'] = sampled_df['trip_distance'] * unit_rev_loss
    sampled_df = sampled_df.reset_index(drop = True)
    
    return sampled_df

In [12]:
def evaluate(chain, c, n):
    O_ID = 2 * n
    D_ID = 2 * n + 1
    
    cost = 0
    for ch in chain:
        cost += c[O_ID, ch[0] - 1]
        for i in range(len(ch) - 1):
             cost += c[ch[i] - 1, ch[i] - 1 + n]
             if ch[i + 1] != 'd':
                 cost += c[ch[i] - 1 + n, ch[i + 1] - 1]
             else:
                 cost += c[ch[i] - 1 + n, D_ID]
    return cost

In [45]:
def phase_1_sampling(order_cnt, iter_time):
    # parameters
    pi = 100  # unit revenue losss
    fc = 100  # fleet_cost
    dc = 30  # dispatch_cost
    rc = 5  # relocation_cost
    max_buffer = 99999
    F = 10000
    temp_c=[]
    temp_relocation_time=[]
    temp_df_sampled =[]
    temp_chain=[]
    temp_cost=[]
    temp_alg_time=[]

    for i in range(iter_time):
        c, pick_up_time, deliver_time, relocation_time, df_sampled =\
            generate_an_instance(n = order_cnt, unit_rev_loss = pi, fleet_cost = fc, dispatch_cost = dc, relocation_cost = rc)
        s_t = time.time()
        chain, cur_cost = sol_pro(c, F, pick_up_time, deliver_time, avg_distance_dict, relocation_time, max_buffer)
        e_t = time.time()
        alg_time = e_t - s_t
        temp_c.append(c)
        temp_relocation_time.append(relocation_time)
        temp_df_sampled.append(df_sampled)
        temp_chain.append(chain)
        temp_cost.append(cur_cost)
        temp_alg_time.append(alg_time)

    return temp_c, temp_relocation_time, temp_df_sampled, temp_chain, temp_cost, temp_alg_time


In [46]:
def run_an_experiment(order_cnt, random_order_cnt, c, relocation_time, df_sampled, chain, cur_cost, alg_time):    
    # parameters
    pi = 100  # unit revenue losss
    fc = 100  # fleet_cost
    dc = 30  # dispatch_cost
    rc = 5  # relocation_cost
    max_buffer = 99999
    #total_cnt = order_cnt + random_order_cnt
    
    # phase 1
    # ==========
    F = 10000
    #c, pick_up_time, deliver_time, relocation_time, df_sampled =\
    #    generate_an_instance(n = order_cnt, unit_rev_loss = pi, fleet_cost = fc, dispatch_cost = dc, relocation_cost = rc)
    #s_t = time.time()
    #chain, cur_cost = sol_pro(c, F, pick_up_time, deliver_time, avg_distance_dict, relocation_time, max_buffer)
    #e_t = time.time()
    #alg_time = e_t - s_t
    # ==========
    
    # phase 2
    # ==========
    for i in range(random_order_cnt):
        df_random_order = generate_a_random_order(unit_rev_loss = pi)
        df_sampled.loc[len(df_sampled)] = df_random_order.loc[0]
        df_sampled = df_sampled.reset_index(drop = True)
        s_time, e_time, s_loc, e_loc, rev_loss = get_basic_info(df_sampled)
        
        s_t = time.time()
        pick_chain, pos, min_cost, cur_cost =\
            add_ord(chain, 
                    df_random_order.loc[0, 'tpep_pickup_datetime'], df_random_order.loc[0, 'tpep_dropoff_datetime'], 
                    s_time, e_time, s_loc, e_loc,
                    df_random_order.loc[0, 'PULocationID'], df_random_order.loc[0, 'DOLocationID'], 
                    avg_distance_dict, relocation_time, rc, fc, dc, cur_cost, df_random_order.loc[0, 'rev_loss'])
        e_t = time.time()
        alg_time += e_t - s_t
        
        if pick_chain != 'new_chain':
            chain[pick_chain].insert(pos, order_cnt + i + 1)
        else:
            chain.append([order_cnt + i + 1, 'd'])
    # ==========
    
    # optimal solution
    # ==========
    s_time, e_time, s_loc, e_loc, rev_loss = get_basic_info(df_sampled)
    c, relocation_time = get_all_info(len(df_sampled), fc, dc, s_loc, e_loc, rev_loss, rc)
    s_t = time.time()
    opt_chain, opt_cost = sol_pro(c, F, s_time, e_time, avg_distance_dict, relocation_time, max_buffer)
    e_t = time.time()
    opt_time = e_t - s_t
    # ==========
    if cur_cost<opt_cost:
        print(df_sampled)
        print(chain)
        print(opt_chain)
        print(cur_cost, opt_cost)
    # print(f'ALG: cost = {evaluate(chain, c, total_cnt)}, chain = {chain}')
    # print(f'OPT: cost = {evaluate(opt_chain, c, total_cnt)}, chain = {opt_chain}')

    return cur_cost, opt_cost, alg_time, opt_time, c, relocation_time, df_sampled, chain, cur_cost, alg_time

In [61]:
pi = 100  # unit revenue losss
fc = 100  # fleet_cost
dc = 30  # dispatch_cost
rc = 5  # relocation_cost
max_buffer = 99999
F = 10000

s_time, e_time, s_loc, e_loc, rev_loss = get_basic_info(exp_df_sampled[9])
c, relocation_time = get_all_info(len(exp_df_sampled[9]), fc, dc, s_loc, e_loc, rev_loss, rc)
s_t = time.time()
opt_chain, opt_cost = sol_pro(c, F, s_time, e_time, avg_distance_dict, relocation_time, max_buffer)
e_t = time.time()
opt_time = e_t - s_t
print(opt_time)

3.281036138534546


In [63]:
result_df = pd.DataFrame(columns = ['order_cnt', 'random_order_cnt', 'exper_id', 'alg_cost', 'opt_cost',  'alg_time', 'opt_time', 'gap'])

init_order_cnt=100
exper_cnt = 3
order_cnt = 100
exp_random_order_cnt=100
random_order_cnt=100

exp_c, exp_relocation_time, exp_df_sampled, exp_chain, exp_cost, exp_alg_time=phase_1_sampling(init_order_cnt,exper_cnt)

for i in range(3):
    for exper_id in range(exper_cnt):
        print(exper_id, order_cnt, random_order_cnt)
        alg_cost, opt_cost, alg_time, opt_time, exp_c[exper_id], exp_relocation_time[exper_id], exp_df_sampled[exper_id], exp_chain[exper_id], exp_cost[exper_id], exp_alg_time[exper_id] = run_an_experiment(order_cnt, exp_random_order_cnt, exp_c[exper_id], exp_relocation_time[exper_id], exp_df_sampled[exper_id], exp_chain[exper_id], exp_cost[exper_id], exp_alg_time[exper_id])
        
        result_df.loc[len(result_df)] = {
            'order_cnt': init_order_cnt,
            'random_order_cnt': (i+1)*exp_random_order_cnt,
            'exper_id': exper_id,
            'alg_cost': alg_cost,
            'opt_cost': opt_cost,
            'alg_time': alg_time,
            'opt_time': opt_time,
            'gap': -1*(alg_cost-opt_cost)/opt_cost
        }
        
        result_df.to_excel('./testt.xlsx', index = False)
    order_cnt+=exp_random_order_cnt

0 100 100


/var/folders/68/t3g8pbgd4f56sf_dcgl_pc140000gn/T/ipykernel_34880/1126366453.py:28: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  result_df.to_excel('./testt.xlsx', index = False)


1 100 100


/var/folders/68/t3g8pbgd4f56sf_dcgl_pc140000gn/T/ipykernel_34880/1126366453.py:28: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  result_df.to_excel('./testt.xlsx', index = False)


2 100 100


...

/var/folders/68/t3g8pbgd4f56sf_dcgl_pc140000gn/T/ipykernel_34880/1126366453.py:28: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  result_df.to_excel('./testt.xlsx', index = False)


1 300 100


/var/folders/68/t3g8pbgd4f56sf_dcgl_pc140000gn/T/ipykernel_34880/1126366453.py:28: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  result_df.to_excel('./testt.xlsx', index = False)


2 300 100


/var/folders/68/t3g8pbgd4f56sf_dcgl_pc140000gn/T/ipykernel_34880/1126366453.py:28: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  result_df.to_excel('./testt.xlsx', index = False)


In [58]:
gap_mean=result_df.groupby(['random_order_cnt'])['gap'].mean()
alg_time_mean=result_df.groupby(['random_order_cnt'])['alg_time'].mean()
opt_time_mean=result_df.groupby(['random_order_cnt'])['opt_time'].mean()

df_summary_result = pd.DataFrame({'gap_mean': gap_mean, 'alg_time_mean': alg_time_mean, 'opt_time_mean': opt_time_mean})
df_summary_result.to_excel('./summary_stat.xlsx', index = False)


/var/folders/68/t3g8pbgd4f56sf_dcgl_pc140000gn/T/ipykernel_34880/1121800300.py:6: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df_summary_result.to_excel('./summary_stat.xlsx', index = False)
